In [1]:
import pandas as pd
import sqlalchemy as sa
import pywikidata as wp

In [2]:
df_wiki = pd.read_csv('sample_data/query_cities_population_gps.csv')
df_wiki

,cityLabel,population,gps
0,Edo,http://www.wikidata.org/.well-known/genid/87cd...,Point(139.774444444 35.683888888)
1,Barxelas,0,Point(-7.845761111 42.370038888)
2,A Garduñeira,0,Point(-7.968938888 42.340813888)
3,Q61379203,0,Point(-7.481878 42.713333)
4,Q20546571,0,Point(-7.679743 42.339763)
...,...,...,...
38996,Chengdu,20937757,Point(104.063333333 30.66)
38997,Beijing,21893095,Point(116.407526 39.90403)
38998,Shanghai,24870895,Point(121.466666666 31.166666666)
38999,Delhi,26495000,Point(77.216666666 28.666666666)


In [3]:
# MusicBrainz database credentials
HOST = "localhost"
DATABASE = "musicbrainz_db"
USER = "musicbrainz"
PORT = 5432

sa_conn_str = f"postgresql+psycopg2://{USER}@{HOST}:{PORT}/{DATABASE}"
engine = sa.create_engine(sa_conn_str)
# conn = engine.connect()

In [4]:
with engine.connect() as conn:
    df_musicbrainz = pd.read_sql(sa.text("SELECT name from area"), conn)

df_musicbrainz

,name
0,Greccio
1,Canada
2,Chile
3,China
4,Cambodia
...,...
118948,Lomma
118949,Mellunmäki
118950,Munkkiniemi
118951,Simonswood


In [5]:
df_musicbrainz['name'] = df_musicbrainz['name'].str.lower()
df_wiki['cityLabel'] = df_wiki['cityLabel'].str.lower()

not_in_MeB = df_wiki[~df_wiki.cityLabel.isin(df_musicbrainz.name)]
is_in_MeB = df_wiki[df_wiki.cityLabel.isin(df_musicbrainz.name)]

In [6]:
not_in_MeB

,cityLabel,population,gps
1,barxelas,0,Point(-7.845761111 42.370038888)
2,a garduñeira,0,Point(-7.968938888 42.340813888)
3,q61379203,0,Point(-7.481878 42.713333)
4,q20546571,0,Point(-7.679743 42.339763)
5,lousadela,0,Point(-7.238972222 42.604722222)
...,...,...,...
38874,saint petersburg,5384342,Point(30.316666666 59.95)
38889,pudong,5681512,Point(121.539722222 31.223055555)
38944,new york city,8804190,Point(-74.0 40.7)
38953,mexico city,9209944,Point(-99.145555555 19.419444444)


In [48]:
not_in_MeB[not_in_MeB.population.astype(float) < 1000]

,cityLabel,population,gps
1,barxelas,0,Point(-7.845761111 42.370038888)
2,a garduñeira,0,Point(-7.968938888 42.340813888)
3,q61379203,0,Point(-7.481878 42.713333)
4,q20546571,0,Point(-7.679743 42.339763)
5,lousadela,0,Point(-7.238972222 42.604722222)
...,...,...,...
9775,buduslău,995,Point(22.262462 47.400931)
9783,bârla,996,Point(24.76666667 44.41666667)
9784,comăna de jos,996,Point(25.231288 45.913745)
9790,grințieș,997,Point(25.883 47.033)
